# Constructing dense offset maps from Sentinel-1 SLC's
<br>

**Author:** Jack Logan <br>
**Last Revised:** December 7, 2025

This notebook uses the ISCE2 script `topsApp.py` to generate dense offset maps (pixel tracking). 
This notebook is a step-by-step example for generating one offset from a pair of SLC's using the code from `run_isce2.py`. 
The script is used to run ISCE2 processing on multiple consecutive images from the command line. Follow this notebook for the first offset to understand the process. Then, edit the XML parameters in `run_isce2.py`, and run the rest from the command line.

> Refer to the UNAVCO Tutorial from [2021](https://github.com/parosen/Geo-SInC/blob/main/UNAVCO2021/4.4_Offset_stack_for_velocity_dynamics/nb_topsApp_offsets.ipynb) for a conceptual understanding of pixel tracking.

**Step 1:** Get Sentinel-1 SLC's from ASF <br>
**Step 2:** Get orbits files <br>
**Step 3:** Write XML files <br>
**Step 4:** Run `topsApp.py`<br>

## Import Packages

In [ ]:
import os
import subprocess
from datetime import datetime

import asf_search as asf
import pandas as pd
import geopandas as gpd
import getpass

from shapely.geometry import Polygon
from glob import glob

os.chdir('offsets/')  # Move to offsets directory

## Step 1: Getting Sentinel-1 Data

Below, we will downloads Sentinl-1 SLC bursts from Alaska Satellite Facility (ASF) using the package `asf_search`. 
For this example, we are going to generate offsets using the images from August 7, 2018 and August 19, 2018. 

When running from the command line, you will be prompted to enter the number of images to download and the date you would like to start from. You can easily adapt the script to your own AOI by changing lines 26 and 34 with your own:

```python
    # . . Change this WKT to your desired area
    aoi = 'POLYGON((38.0336 -69.7358,38.0336 -70.4952,39.6985 -70.4952,39.6985 -69.7358,38.0336 -69.7358))'
    
    n_files = int(input('Number of files to download: '))
    if n_files:
        opts = {
            'platform':'S1',
            'start':str(input('Start Date (YYYY-MM-DD): ')),
            'processingLevel':'SLC',
            'frame':[830, 834, 936, 938, 939],  # Either remove this line or replace with your ASF frames
        }
        results = asf.search(intersectsWith=aoi, **opts)[-n_files:]
```

In [ ]:
# . . Shirase Glacier WKT for asf_search
aoi = 'POLYGON((38.0336 -69.7358,38.0336 -70.4952,39.6985 -70.4952,39.6985 -69.7358,38.0336 -69.7358))'
n_files = 2  # Only keep two most recent scenes for demo (command line input)

# . . Parameters to restrict search
opts = {
    'platform':'S1',
    'start':'2018-08-07',               # Starting date for dem (command line input)
    'processingLevel':'SLC',
    'frame':[830, 834, 936, 938, 939],  # Scenes known to fully cover Shirase Glacier
}
results = asf.search(intersectsWith=aoi, **opts)[-n_files:]

### Start ASF Session and Download the Zip files

Enter your NASA EarthData login credentials here to authenticate and the files will be downloaded.

In [ ]:
session = asf.ASFSession()

username = input('Username:')
password = getpass.getpass('Password:')

try:
    user_pass_session = asf.ASFSession().auth_with_creds(username, password)
except asf.ASFAuthenticationError as e:
    print(f'Auth failed: {e}')
else:
    print('Success!')

results.download(path='SAFE/', session=user_pass_session)

## Step 2: Orbit Files

For dense offsets, we need the precise orbits for each pass. This is will allow ISCE2 to accurately coregister the images and compute the dense offsets.

In [ ]:
# . . Get name of all safe files
safe_files = glob(f'./SAFE/*.zip')

# . . Download Orbit files to orbit folder
os.chdir(f'./orbits')
for file in safe_files:
    os.system(f'../fetchOrbit.py -i {file[5:-5]}')

## Step 3: XML Files

The XML files tell ISCE2 where to find all of the input files, and what steps to run when processing. The reference and secondary give file paths to the image pairs and orbit data.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<component name="reference">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180807T175609_20180807T175636_023143_028391_AE45.zip</property>
    <property name="output directory">reference</property>
    <property name="orbit directory">../orbits</property>
    <property name="polarization">hh</property>
</component>
```

```xml
<?xml version="1.0" encoding="UTF-8"?>
<component name="secondary">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180819T175610_20180819T175637_023318_02893F_6998.zip</property>
    <property name="output directory">secondary</property>
    <property name="orbit directory">../orbits</property>
    <property name="polarization">hh</property>
</component>
```

The main XML, `topsApp.xml`, specifies all the processing steps. We set all InSAR parameters off since we won't be able to unwrap an interferogram in this area. The final block is where we specify the offset parameters.

- `Ampcor window height/width`: Size of the patch to correlate between two images
- `Ampcor search height/width`: How far to look around the patches
- `Ampcor skip height/width`: How far to move before picking a new patch

> Note: The specific height/width ratio is dependent on your geometry and the maximum expected displacement between two passes.
> Because the range resolution is ~4 time higher than the azimuth resolution, your window widths are typically 3 to 6 times larger than the heights.

```xml
<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
    <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>

        <!-- Scene XML files -->
        <component name="reference">
            <catalog>reference.xml</catalog>
        </component>
        <component name="secondary">
            <catalog>secondary.xml</catalog>
        </component>

        <!-- The swaths to process -->
        <property name="swaths">[2]</property>

        <!-- The region of interest -->
        <property name="region of interest">[-70.45441464, -69.88490745, 38.29553816,  39.83817435]</property>

        <!-- DEM for processing -->
        <property name="demFilename">../dem/dem.tiff</property>

        <!-- Unset all InSAR processing steps -->
        <property name="do interferogram">False</property>
        <property name="do ESD">False</property>
        <property name="do unwrap">False</property>
        <property name="do unwrap 2 stage">False</property>
        <property name="do ionosphere correction">False</property>
        <property name="geocode list">[]</property>

        <!-- Parameters for dense offsets -->
        <property name="do denseoffsets">True</property>
        <property name="Ampcor window width">256</property>         <!-- Correlation window width  -->
        <property name="Ampcor window height">64</property>         <!-- Correlation window height -->
        <property name="Ampcor search window width">30</property>   <!-- Horizontal area to search -->
        <property name="Ampcor search window height">10</property>  <!-- Vertical area to search   -->
        <property name="Ampcor skip width">44</property>            <!-- Horizontal area to jump -->
        <property name="Ampcor skip height">8</property>            <!-- Vertical area to jump   -->

    </component>
</topsApp>
```

### Create offset specific directory, write XML files

Now we create a directory for the displacement between these two dates. This will contain all the ISCE2 output files to one folder and reduce clutter in the main directories. 

After making the directory, we move into the and write the XML input files inside.

In [ ]:
# . . Make directory, overwrite if existing
os.makedirs('../20180807-20180819/', exist_ok=True)

# . . Move into directory
os.chdir('../20180807-20180819/')

# . . XML file strings
topsApp = """<?xml version="1.0" encoding="UTF-8"?>
<topsApp>
    <component name="topsinsar">
        <property name="Sensor name">SENTINEL1</property>

        <!-- Scene XML files -->
        <component name="reference">
            <catalog>reference.xml</catalog>
        </component>
        <component name="secondary">
            <catalog>secondary.xml</catalog>
        </component>

        <!-- The swaths to process -->
        <property name="swaths">[2]</property>

        <!-- The region of interest -->
        <property name="region of interest">[-70.45441464, -69.88490745, 38.29553816,  39.83817435]</property>

        <!-- DEM for processing -->
        <property name="demFilename">../dem/fixed_dem.wgs84</property>

        <!-- Unset all InSAR processing steps -->
        <property name="do interferogram">False</property>
        <property name="do ESD">False</property>
        <property name="do unwrap">False</property>
        <property name="do unwrap 2 stage">False</property>
        <property name="do ionosphere correction">False</property>
        <property name="geocode list">[]</property>

        <!-- Parameters for dense offsets -->
        <property name="do denseoffsets">True</property>
        <property name="Ampcor window width">64</property>
        <property name="Ampcor window height">16</property>
        <property name="Ampcor search window width">128</property>
        <property name="Ampcor search window height">32</property>
        <property name="Ampcor skip width">256</property>
        <property name="Ampcor skip height">64</property>

    </component>
</topsApp>"""

#################################################################################
                     #### Revert window and skip sizes ####
#################################################################################

reference = """<?xml version="1.0" encoding="UTF-8"?>
<component name="reference">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180807T175609_20180807T175636_023143_028391_AE45.zip</property>
    <property name="output directory">reference</property>
    <property name="orbit directory">../orbits</property>
    <property name="polarization">hh</property>
</component>"""

secondary = """<?xml version="1.0" encoding="UTF-8"?>
<component name="secondary">
    <property name="safe">../SAFE/S1A_IW_SLC__1SSH_20180819T175610_20180819T175637_023318_02893F_6998.zip</property>
    <property name="output directory">secondary</property>
    <property name="orbit directory">../orbits</property>
    <property name="polarization">hh</property>
</component>"""

# . . Write XML input files
with open('topsApp.xml', 'w') as f:
    f.write(topsApp)

with open('reference.xml', 'w') as f:
    f.write(reference)

with open('secondary.xml', 'w') as f:
    f.write(secondary)

## Step 4: Run topsApp.py

To run `topsApp` from the command line, we have to fix some environment variables. 
The following code will have to be run in any python script before calling any ISCE scripts from the command line.
This code can be found in the `setup_environment` function in `run_isce2.py`. To properly setup the environment on your machine, change lines 82 and 83:
```python
def setup_environment():
    """Setup ISCE2 environment"""
    conda_path = '/home/jovyan'  # Your anaconda directory
    isce_env = 'envs/isce2'      # Where your ISCE2 environment is saved
```

In [ ]:
# . . Root paths
conda_path = '/home/jovyan'  # Where all anaconda environments are saved
isce_env = 'envs/isce2'      # ISCE2 environment created in Notebook 1

isce_home = f'{conda_path}/{isce_env}/lib/python3.8/site-packages/isce'
isce_stack = f'{conda_path}/{isce_env}/share/isce2'
isce_lib_path = f"{conda_path}/{isce_env}/lib"

os.environ["LD_LIBRARY_PATH"] = f"{isce_lib_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"
os.environ['ISCE_HOME'] = isce_home
os.environ['ISCE_STACK'] = isce_stack
os.environ['ISCE_ROOT'] = f'{conda_path}/{isce_env}/lib/python3.8/site-packages'

# Update PATH and PYTHONPATH
path_components = [
    f"{isce_home}/bin",
    f"{isce_home}/applications",
    f"{isce_stack}/topsStack",
    os.environ.get('PATH', '')
]
os.environ['PATH'] = ':'.join(filter(None, path_components))

pythonpath_components = [
    f'{conda_path}/{isce_env}/lib/python3.8/site-packages',
    isce_home,
    isce_stack,
    f"{isce_home}/applications",
    f"{isce_home}/components",
    os.environ.get('PYTHONPATH', '')
]
os.environ['PYTHONPATH'] = ':'.join(filter(None, pythonpath_components))

os.environ['OMP_NUM_THREADS'] = '8'

Now we run this cell and wait. 
With the default parameters for this example, the runtime is ~2 hours per offset.

In [ ]:
cmd = f"conda run -n isce2 topsApp.py topsApp.xml"

process = subprocess.Popen(
    cmd, 
    shell=True, 
    text=True,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    env=os.environ,
    cwd=os.getcwd(),
)
while True:
    output = process.stdout.readline()
    if output == '' and process.poll() is not None:
        break
    if output:
        message = output.strip()
        print(message)

## Step 6: Automate Offsets

Now that we've gone through this process, we can run the python script `run_isce2.py` from the command line. 
It will prompt you to input your NASA EarthData credentials, the number of SAFE files to download, and a start date. 
It will then automatically download the Orbit files required and begin the processing. 

In the event there is a missing SLC, creating a 24 day repeat pass, the script automatically accounts for thie change and doubles the search window sizes. 
We recommend limiting your search windows to just larger than the maximum observed or expected displacement, found empirically.

After each offset is computed, we delete all the extra files left over by the processing, then clip the data to your ROI. Unfortunately this last step is hardcoded, and requires another change in values.

**To customize for your own region:**
1. Change the AOI wicket on line 26 and the frames on line 34
3. Adjust the `region of interest` property on line 281.
4. Change the Ampcor window properties on lines 295-301.
5. Change the clipping limits in line 396. <br>
   *Note the decreasing y-axis when slicing*